In [1]:
pip install dash pandas


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
import dash
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
from threading import Timer
from flask import request

# Load the dataset
df = pd.read_csv('../world_happiness_2021.csv')

# List of columns to use as options for x-axis
x_columns = [
    "GDP per capita", 
    "Social support", 
    "Healthy life expectancy", 
    "Freedom to make life choices", 
    "Generosity", 
    "Perceptions of corruption"
]

# Create a Dash app
app = Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("World Happiness 2021"),
    
    # Dropdown to select the column for x-axis
    dcc.Dropdown(
        id='x-axis-dropdown',
        options=[{'label': col, 'value': col} for col in x_columns],
        value=x_columns[0],  # Default value
        clearable=False,
        style={'width': '50%'}
    ),
    
    # Graph to display the plot
    dcc.Graph(id='scatter-plot')
])

# Callback to update the graph based on selected x-axis column
@app.callback(
    Output('scatter-plot', 'figure'),
    Input('x-axis-dropdown', 'value')
)
def update_graph(selected_x):
    # Create the scatter plot based on selected x-axis column
    fig = px.scatter(
        df,
        x=selected_x,
        y="Ladder score",
        color="Regional indicator",
        size="Social support",  # Optional: size can be used to add more detail
        hover_name="Country name",
        template='simple_white',
        title=f"Scatter Plot of Ladder Score vs {selected_x}")
     # Update layout to make the graph square
    fig.update_layout(
        autosize=False,
        width=800,  # Set the width of the graph
        height=600  # Set the height of the graph (same as width for square)
    
    )
    return fig


# Function to stop the server
def shutdown_server():
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()

# Start a timer to shut down the app after 5 seconds
def schedule_shutdown():
    Timer(5, shutdown_server).start()

# Run the server on a different port (e.g., 8080)
if __name__ == '__main__':
    schedule_shutdown()
    app.run_server(debug=True, port=8081)
